In [1]:
"""
@author: pho
"""
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False
import sys
import importlib
import numpy as np
import pandas as pd
from pathlib import Path
import h5py
import hdf5storage # conda install hdf5storage

from neuropy.utils.dynamic_container import DynamicContainer
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatBaseRegisteredClass
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.core import DataWriter, NeuronType, Neurons, BinnedSpiketrain, Mua, ProbeGroup, Position, Epoch, Signal, Laps, FlattenedSpiketrains
from neuropy.core.session.dataSession import DataSession


from pyphocorehelpers.print_helpers import print_keys_if_possible, debug_dump_object_member_shapes
# # import PhoPositionalData as pdp
# # from pyphoplacecellanalysis.PhoPositionalData. import load_exported, process_data
# from pyphoplacecellanalysis.PhoPositionalData.load_exported import *
# # from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from pyphoplacecellanalysis.PhoPositionalData.process_data import *
# from pyphoplacecellanalysis.PhoPositionalData.plot_data import *
# from pyphoplacecellanalysis.PhoPositionalData.import_data import *
# from pyphoplacecellanalysis.PhoPositionalData.load_exported import import_mat_file
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, gen_2d_histrogram, get_heatmap_color_vectors, process_chunk_equal_poritions_data, extract_spike_timeseries
# from pyphoplacecellanalysis.PhoPositionalData.process_data import process_positionalAnalysis_data, extract_spike_timeseries

In [ ]:
## Drastically improve Python Tracebacks on exceptions:
# from rich.traceback import install
# install()

## Automatic debugger on exception raised
%pdb on

In [2]:
# RoyMaze1:
# mat_import_parent_path = Path(r'R:\data\RoyMaze1')
mat_import_parent_path = Path(r'R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1')
# mat_import_parent_path = Path(r'C:\Share\data\RoyMaze1') # Old one
# mat_import_file = r'C:\Share\data\RoyMaze1\ExportedData.mat'

In [3]:
session = HiroDataSessionFormatRegisteredClass.get_session(basedir=mat_import_parent_path)
session

c:\users\pho\repos\neuropy\neuropy\core\session\Formats\SessionSpecifications.py:139: UserWarning: WARNING: Optional File: R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1\RoyMaze1.xml does not exist. Continuing without it.
  warnings.warn(f'WARNING: Optional File: {an_optional_filepath} does not exist. Continuing without it.')


Loading matlab import file results to R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1\ExportedData\positionAnalysis.mat... done.
Loading matlab import file results to R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1\ExportedData\spikesAnalysis.mat... done.
Loading matlab import file results to R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1\ExportedData\extrasAnalysis.mat... done.
num_periods: 668
Failure loading .position.npy. Must recompute.

Computing linear positions for all active epochs for session... Saving updated position results results to R:\rMBP Python Repos 2022-07-07\PhoNeuronGillespie2021CodeRepo\PhoMatlabDataScripting\ExportedData\RoyMaze1\RoyMaze1.position.npy... RoyMaze1.position.npy saved
done.
Loading success: .flattened.spikes.npy.
Failure loading .ripple.npy. Must recompute.

computing ri

DataSession(configured from manual recinfo: DynamicContainer({'source_file': None, 'channel_groups': None, 'skipped_channels': None, 'discarded_channels': None, 'n_channels': None, 'dat_sampling_rate': 30000.0, 'eeg_sampling_rate': 1250.0}))

In [ ]:
session.epochs

In [ ]:
session.position

In [ ]:
session.epochs.labels

In [ ]:
# Common Extended properties:
session = HiroDataSessionFormatRegisteredClass._default_extended_postload(session.filePrefix, session)
session

In [ ]:
active_file_suffix = '.flattened.spikes.npy'
active_file_suffix = '.flattened.spikes.npy'
found_datafile = session.filePrefix.with_suffix(active_file_suffix)
print(f'found_datafile: {str(found_datafile)} -- exists? {found_datafile.exists()}')

# if found_datafile is not None:
#     print('Loading success: {}.'.format(active_file_suffix))
#     session.flattened_spiketrains = found_datafile
# else:
#     # Otherwise load failed, perform the fallback computation
#     print('Failure loading {}. Must recompute.\n'.format(active_file_suffix))

In [ ]:
# Load or compute linear positions if needed:
session = HiroDataSessionFormatRegisteredClass._default_compute_linear_position_if_needed(session)
# debug_print = True
# # compute linear positions:
# print('Computing linear positions for all active epochs for session...', end=' ')
# # end result will be session.computed_traces of the same length as session.traces in terms of frames, with all non-maze times holding NaN values
# session.position.linear_pos = np.full_like(session.position.time, np.nan)
# for anEpochLabelName in session.epochs.labels:
#     if anEpochLabelName in ['track']:
#         curr_active_epoch_timeslice_indicies, active_positions_maze1, linearized_positions_maze1 = DataSession.compute_linearized_position(session, epochLabelName=anEpochLabelName, method='pca')
#         # session.position.linear_pos[curr_active_epoch_timeslice_indicies] = linearized_positions_maze1.traces
#         if debug_print:
#             print('\t curr_active_epoch_timeslice_indicies: {}\n \t np.shape(curr_active_epoch_timeslice_indicies): {}'.format(curr_active_epoch_timeslice_indicies, np.shape(curr_active_epoch_timeslice_indicies)))

#         session.position._data.loc[curr_active_epoch_timeslice_indicies, 'lin_pos'] = linearized_positions_maze1.x
#     else:
#         if debug_print:
#             print(f'\t skipping non-maze epoch "{anEpochLabelName}"')

In [ ]:
session.position

## Previous Notebook Session-loading implementation before refactoring into HiroDataSessionFormat.py

In [ ]:
session = HiroDataSessionFormatRegisteredClass.build_session(basedir=mat_import_parent_path)
session

In [ ]:
session, loaded_file_record_list = DataSessionFormatBaseRegisteredClass.load_session(session, debug_print=True) # call the super class load_session(...) to load the common things (.recinfo, .filePrefix, .eegfile, .datfile)

In [ ]:
# all_vars = load_position_spikes_extras_mats(parent_path=mat_import_parent_path)
all_vars = HiroDataSessionFormatRegisteredClass._load_all_mats(parent_path=mat_import_parent_path)
# all_vars

In [ ]:
# all_vars.extras
# print_keys_if_possible('pos', all_vars.pos)
print_keys_if_possible('extras', all_vars.extras)
# print_keys_if_possible('all_vars', all_vars)
# debug_dump_object_member_shapes(all_vars)

In [ ]:
type(all_vars.extras.behavioral_epochs) # pandas.core.frame.DataFrame
print(list(all_vars.extras.behavioral_epochs.columns)) # ['epoch_index', 'start_seconds_absolute', 'end_seconds_absolute', 'start_seconds', 'end_seconds', 'duration']
all_vars.extras.behavioral_epochs

In [ ]:
## Adds Session.paradigm (Epochs)
session_absolute_start_timestamp = all_vars.extras.behavioral_epochs.loc[0, 'start_seconds_absolute'] # 68368.714228
session.config.absolute_start_timestamp = session_absolute_start_timestamp
# 'start_seconds' and 'end_seconds' are relative to start
if 'label' in all_vars.extras.behavioral_epochs.columns:
    epoch_labels = all_vars.extras.behavioral_epochs['label']
else:
    num_rows = all_vars.extras.behavioral_epochs.shape[0]
    epoch_labels = [f'epoch{i}' for i in np.arange(num_rows)]
epochs_df = pd.DataFrame({'label':epoch_labels, 'start':all_vars.extras.behavioral_epochs['start_seconds'].to_numpy(),'stop':all_vars.extras.behavioral_epochs['end_seconds'].to_numpy()})
session.paradigm = Epoch(epochs=epochs_df)
session.paradigm

In [ ]:
## Adds Positions:
position_sampling_rate_Hz = 1.0 / np.nanmean(np.diff(all_vars.pos.t)) # 1.0/0.03336651239320582 = 29.97016853950917 Hz
session.config.position_sampling_rate_Hz = position_sampling_rate_Hz
# session.position = Position(traces=np.vstack((x, y)), computed_traces=np.full([1, num_samples], np.nan), t_start=active_t_start, sampling_rate=position_sampling_rate_Hz)
t_rel = all_vars.pos.t
x = all_vars.pos.x
y = all_vars.pos.y
session.position = Position.from_separate_arrays(t_rel, x, y)
session.position

In [ ]:
## Adds Spikes:
print(list(all_vars.spikes.keys())) # ['spike_matrix', 'spike_cells', 'num_cells', 'spike_list', 'spike_positions_list', 'flat_cell_ids', 'reverse_cellID_idx_lookup_map', 'spikes_cell_info_out_dict']

In [ ]:
all_vars.spikes.spike_list[0].shape # (293076,)
all_vars.spikes.spike_list[1].shape # (9297,)

# len(all_vars.spikes.spike_list) # 126

In [ ]:
session.paradigm.t_stop

In [ ]:
# all_vars.spikes
# 'spike_cells'
all_vars.spikes.spikes_cell_info_out_dict

In [ ]:
flat_cell_ids = all_vars.spikes.spikes_cell_info_out_dict.aclu
cell_type = NeuronType.from_qclu_series(qclu_Series=all_vars.spikes.spikes_cell_info_out_dict.qclu)
shank_ids = all_vars.spikes.spikes_cell_info_out_dict.shank
# cluster_ids = all_vars.spikes.spikes_cell_info_out_dict.cluster # NOT USED

In [ ]:
spiketrains = np.array(all_vars.spikes.spike_list, dtype='object')
# t_stop = np.max(flat_spikes_out_dict[time_variable_name])
t_stop = session.paradigm.t_stop
flat_cell_ids = np.array(flat_cell_ids)
# all_vars.spikes.spikes_cell_info_out_dict.speculated_unit_type

dat_sampling_rate = 30000.0
lfpSampleRate = 1250.0
posSampleRate = 29.9700

session.neurons = Neurons(spiketrains, t_stop, t_start=0,
    sampling_rate=dat_sampling_rate, # session.recinfo.dat_sampling_rate
    neuron_ids=flat_cell_ids,
    neuron_type=cell_type,
    shank_ids=shank_ids
)
session.neurons

# Neurons
#  n_neurons: 126
#  n_total_spikes: 11214527
#  t_start: 0
#  t_stop: 35351.059333

# 'num_cells': 86, 'spike_list', 'reverse_cellID_idx_lookup_map'

In [ ]:
## IDEA: Define a series of fallback options for each variable using the command chain pattern.